# Parameter Inference for ODE Models: pypesto + AMICI + PEtab

## Motivation: Why Mathematical Modeling?

**Case Study**: Single-cell mRNA translation kinetics after transfection

### The Biological Question

When cells are transfected with mRNA (e.g., for therapeutic applications like mRNA vaccines), we observe:
- Time-lapse fluorescence measurements of protein expression
- **Large cell-to-cell variability** in expression dynamics  
- Different protein variants show different kinetics

### Key Questions Mathematical Modeling Can Answer:

1. **What is the mRNA lifetime?** → Critical for therapeutic dosing design
2. **What causes cell-to-cell variability?** → Biological noise vs. experimental factors
3. **Are ribosomes rate-limiting?** → Mechanistic insight into translation
4. **Can we predict therapeutic efficacy?** → Translation to clinical applications

**Without modeling**: We see fluorescence curves  
**With modeling**: We extract quantitative parameters (rates, lifetimes) and test mechanistic hypotheses

---

### Today's Dataset:

From **Fröhlich et al. (2018)** *npj Systems Biology and Applications*:  
*"Multi-experiment nonlinear mixed effect modeling of single-cell translation kinetics after transfection"*

- **>500 single cells** tracked for 30 hours each
- Two protein variants: **eGFP** (stable) and **d2eGFP** (destabilized via PEST sequence)
- Micropatterned arrays + automated time-lapse microscopy
- Data: https://doi.org/10.5281/zenodo.1228898

---

## The Model: mRNA Translation Kinetics

**Fröhlich et al. Fig. 4a, Model (i)**: Standard two-stage gene expression model

### ODEs:

$$\frac{d[\text{mRNA}]}{dt} = -\delta [\text{mRNA}]$$

$$\frac{d[\text{GFP}]}{dt} = k[\text{mRNA}] - \gamma [\text{GFP}]$$

### Parameters:
- $\delta$: mRNA degradation rate (h⁻¹) → **What we want to estimate!**
- $k$: Translation rate (h⁻¹)
- $\gamma$: Protein degradation rate (h⁻¹) → Different for eGFP vs d2eGFP
- $m_0$: Initial mRNA amount (transfected at time $t_0$)

### Observable:
$$y(t) = \log(\text{scale} \cdot [\text{GFP}](t) + \text{offset})$$

Log-transform handles multiplicative noise in fluorescence measurements.

---

## Setup: Install Required Packages

```bash
pip install pypesto amici petab pandas matplotlib numpy scipy
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tempfile
from scipy.integrate import odeint

# pypesto, AMICI, PEtab (may need to install these)
try:
    import pypesto
    import pypesto.petab
    import pypesto.optimize
    import pypesto.visualize
    import petab
    import amici
    import amici.petab.petab_import
    print("All packages loaded successfully")
except ImportError as e:
    print(f"Missing package: {e}")
    print("Install with: pip install pypesto amici petab")

# For reproducibility
np.random.seed(42)

---

# Part 1: Data Loading

## Step 1: Load Real Experimental Data

We'll use single-cell fluorescence trajectories from **Fröhlich et al. (2018)**:
- **eGFP dataset**: Stable protein (low degradation, $\gamma \approx 0.01$ h⁻¹)
- **d2eGFP dataset**: Destabilized protein (PEST sequence, $\gamma \approx 0.3$ h⁻¹)

**Data format** (Excel files from Zenodo):
- `20160427_mean_eGFP.xlsx` and `20160427_mean_d2eGFP.xlsx`
- Column 1: Time (hours)
- Columns 2, 3, 4, ...: Individual cell fluorescence trajectories
- Each row = one time point, each column (after time) = one cell

In [ ]:
# ═══════════════════════════════════════════════════════════════
# FILE INPUT: User provides data files here
# ═══════════════════════════════════════════════════════════════

# Specify data directory
DATA_DIR = './data/'  # ← USER: Adjust this path

# Expected file names from Zenodo (https://doi.org/10.5281/zenodo.1228898)
# Format: Column 1 = time, Column 2+ = individual cells
EGFP_DATA_FILE = os.path.join(DATA_DIR, '20160427_mean_eGFP.xlsx')

# Check if real data exists
USE_REAL_DATA = os.path.exists(EGFP_DATA_FILE)

if USE_REAL_DATA:
    print("✓ Loading REAL experimental data from Fröhlich et al. 2018...\n")
    
    # Load eGFP data (Excel format)
    egfp_df = pd.read_excel(EGFP_DATA_FILE)
    # First column is time, remaining columns are individual cells
    time_points = egfp_df.iloc[:, 0].values  # First column
    egfp_trajectories = egfp_df.iloc[:, 1:].values.T  # Transpose: cells × time
    
    n_cells_egfp = egfp_trajectories.shape[0]
    
    print(f"  eGFP:    {n_cells_egfp} cells × {len(time_points)} time points")
    print(f"  Duration: {time_points[0]:.1f} to {time_points[-1]:.1f} hours")
    print(f"\n  Data structure:")
    print(f"    Column 1: Time (hours)")
    print(f"    Columns 2+: Individual cell fluorescence trajectories")
    
else:
    print("⚠ Real data files NOT found. Using SYNTHETIC data for demonstration.\n")
    print(f"Expected files (Excel format from Zenodo):")
    print(f"  • {EGFP_DATA_FILE}")
    print(f"\nTo use real data:")
    print(f"  1. Download from: https://doi.org/10.5281/zenodo.1228898")
    print(f"  2. Extract files to: {DATA_DIR}")
    print(f"  3. Re-run this cell\n")
    print(f"Expected Excel structure:")
    print(f"  • Column 1: time (hours)")
    print(f"  • Column 2, 3, 4, ...: cell_1, cell_2, cell_3, ... (fluorescence)")
    print("\n" + "="*60)
    print("Proceeding with SYNTHETIC data generation...\n")

## Step 1b: Visualize Single-Cell Heterogeneity

**Key observation**: Both datasets show large cell-to-cell variability, but with distinct dynamics

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

# eGFP (stable protein) - reaches plateau
for i in range(min(30, len(egfp_trajectories))):
    ax.plot(time_points, egfp_trajectories[i], alpha=0.2, color='green', linewidth=0.5)
ax.plot(time_points, egfp_trajectories.mean(axis=0), 'k-', linewidth=3, label='Population mean')
ax.set_xlabel('Time (hours)', fontsize=13)
ax.set_ylabel('Fluorescence (a.u.)', fontsize=13)
ax.set_title('eGFP: Stable Protein\n(Low degradation → Plateau)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()

print("\n📊 Key Observations from Data:")
print("  1. LARGE cell-to-cell variability in both datasets (biological reality!)")
print("  2. eGFP: Rises to plateau → protein is stable (low γ)")
print("  3. d2eGFP: Peaks ~10h then declines → protein degrades (high γ)")
print("\n❓ Critical Question: What is the mRNA lifetime?")
print("  → mRNA degradation rate δ should be THE SAME in both experiments!")
print("  → But can we estimate it from single-cell data?")
print("\n🎯 This is where pypesto + PEtab + AMICI become essential!")

---

# Part 2: PEtab Problem Definition

## Why PEtab?

**PEtab** = Parameter Estimation Tabular format
- **Standardized** format for systems biology parameter estimation
- **Reproducible**: Share your exact problem definition
- **Tool-agnostic**: Works with multiple software (pypesto, COPASI, etc.)
- **Database integration**: Connect to BioModels, Benchmark collection

### Components:
1. **SBML model**: ODE equations (mechanistic model)
2. **Measurement table**: Your experimental data
3. **Observable table**: What you measure (e.g., fluorescence = f(GFP))
4. **Condition table**: Experimental conditions (eGFP vs d2eGFP)
5. **Parameter table**: What to estimate, bounds, priors

## Step 2a: Create SBML Model

We implement the translation model (Fröhlich et al. Fig. 4a Model i) in SBML format.

In [ ]:
# ══════════════════════════════════════════════════════════════
# SBML MODEL: mRNA Translation (Fig 4a Model i)
# ══════════════════════════════════════════════════════════════

# Note: use antimony to generate model and convert to SBML

# Cconvert to SBML
import matplotlib.pyplot as plt
import numpy as np
import tellurium as te

model_str = """
model mRNA_translation
  
  // Species
  species mRNA, Protein; 
  
  // Parameters
  delta_param = 0.2;            // mRNA degradation rate
  k_1 = 5.0;                    // Protein synthesis rate
  gamma_param = 0.05;           // Protein degradation rate (eGFP)
  t_0 = 1.0;                    // Transfection time
  m_0 = 10.0;                   // Initial mRNA amount
  
  // Reactions
  J1: mRNA => ; delta_param * mRNA;
  J2: mRNA => mRNA + Protein; k_1 * mRNA;
  J3: Protein => ; gamma_param * Protein;
  
  // Initial conditions
  mRNA = 0;
  Protein = 0;

  // event to set transfection
  at (time >= t_0): mRNA = m_0;
end
"""

# Load the model
r = te.loada(model_str)

# convert model back to Antimony / SBML
ant_str_before = r.getAntimony()
sbml_str_before = r.getSBML()

petab_dir = './petab_files/'
sbml_file = os.path.join(petab_dir, 'translation_model.xml')
r.exportToSBML(sbml_file, current=False)


print("SBML model created")
print("Reactions:")
print("    • mRNA degradation: mRNA → ∅ (rate δ)")
print("    • Translation: mRNA → mRNA + GFP (rate k)")
print("    • Protein degradation: GFP → ∅ (rate γ)")

## Step 2b: Create Measurement Table

For the **teaching demo**, we'll use a **small subset** of cells to keep computation fast:
- Real data: ~500 cells per condition → Use 10 cells for demo
- Can scale up to full dataset for research

**Note**: In practice, more cells = better parameter estimates!

In [ ]:
# ═══════════════════════════════════════════════════════════════
# SUBSET SELECTION FOR DEMO
# ═══════════════════════════════════════════════════════════════
# For teaching: use small subset (fast computation)
# For research: increase n_cells_demo to use more data

n_cells_demo = 1  # ← USER: Increase for research (e.g., 50, 100, or all cells)

# Select subset
n_cells_available_egfp = len(egfp_trajectories)

n_cells_use_egfp = min(n_cells_demo, n_cells_available_egfp)

# Select random cells for diversity (or take first N for reproducibility)
# Option 1: First N cells (reproducible)
selected_egfp_idx = range(n_cells_use_egfp)

# Option 2: Random cells (uncomment for diversity)
# np.random.seed(42)
# selected_egfp_idx = np.random.choice(n_cells_available_egfp, n_cells_use_egfp, replace=False)
# selected_d2egfp_idx = np.random.choice(n_cells_available_d2egfp, n_cells_use_d2egfp, replace=False)

print(f"Dataset sizes:")
print(f"  Available: eGFP={n_cells_available_egfp} cells")
print(f"  Using for demo: eGFP={n_cells_use_egfp} cells")
print(f"  → Total measurements: {n_cells_use_egfp * len(time_points)}")
print(f"\n💡 Tip: For research, set n_cells_demo = 50-100 for better estimates")

In [ ]:
# Create measurement table in PEtab format
measurements_list = []

# eGFP measurements (selected cells only)
for cell_idx in selected_egfp_idx:
    for t_idx, t in enumerate(time_points):
        fluorescence = egfp_trajectories[cell_idx, t_idx]
        
        measurements_list.append({
            'observableId': 'obs_GFP',
            'simulationConditionId': 'condition_eGFP',
            'measurement': fluorescence,
            'time': t,
            'observableParameters': 'scaling;offset',
            'noiseParameters': 'sigma'
        })

measurement_df = pd.DataFrame(measurements_list)
measurement_file = os.path.join(petab_dir, 'measurements.tsv')
measurement_df.to_csv(measurement_file, sep='\t', index=False)

print(f"\n✓ Measurement table created")
print(f"  Total measurements: {len(measurement_df)}")
print(f"  eGFP: {n_cells_use_egfp} cells × {len(time_points)} time points")
print(f"\n  Example measurements (first 3 rows):")
print(measurement_df.head(3).to_string(index=False))

## Step 2c: Create Observable Table

In [ ]:
# Observable: log(scale*GFP + offset)
observable_df = pd.DataFrame({
    'observableId': ['obs_GFP'],
    'observableFormula': ['observableParameter1_obs_GFP*Protein + observableParameter2_obs_GFP'],
    'noiseFormula': ['noiseParameter1_obs_GFP'],
    'noiseDistribution': ['normal'],
    'observableTransformation': ['lin'],
})

observable_file = os.path.join(petab_dir, 'observables.tsv')
observable_df.to_csv(observable_file, sep='\t', index=False)

print("✓ Observable table created")
print("  Observable: log(scale*GFP + offset)")
print("  → Handles multiplicative noise in fluorescence")

## Step 2d: Create Condition Table

Two conditions differ only in protein degradation rate γ.

In [ ]:
# Conditions: eGFP (stable)
condition_df = pd.DataFrame({
    'conditionId': ['condition_eGFP'],
    'conditionName': ['eGFP_stable'],
    'gamma_param': ['gamma_eGFP']  # Different protein degradation rates
})

condition_file = os.path.join(petab_dir, 'conditions.tsv')
condition_df.to_csv(condition_file, sep='\t', index=False)

print("✓ Condition table created")
print("  condition_eGFP:    γ = gamma_eGFP (low, stable protein)")
print("  → All other parameters (δ, k) shared across conditions!")

## Step 2e: Create Parameter Table

**Key insight**: δ and k are shared; only γ differs between conditions

In [ ]:
# Parameters to estimate
parameter_df = pd.DataFrame({
    'parameterId': ['delta_param', 'k_1', 'gamma_eGFP', 't_0', 'm_0', 'scaling', 'offset', 'sigma'],
    'parameterName': ['mRNA_degradation', 'Translation_rate', 'Protein_deg_eGFP', 
                     'Transfection_time', 'Initial_mRNA_amount', 'scaling_parameter', 'Offset', 'Noise_SD'],
    'parameterScale': ['log10', 'log10', 'log10', 'log10', 'log10', 'lin', 'log10', 'lin'],
    'lowerBound': [1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 1e-5, 0],
    'upperBound': [1e5, 1e5, 1e5, 1e5, 1e5, 1e5, 1e5, 'inf'],
    'nominalValue': [0.1, 1.0, 0.01, 0.3, 5000, 100, 0.1, 0.1],  # Initial guess
    'estimate': [1, 1, 1, 1, 1, 1, 1, 1],  # 1=estimate, 0=fixed
    'parameterType': ['', '', '', '', '', 'scaling', 'offset', 'sigma']
})

parameter_file = os.path.join(petab_dir, 'parameters.tsv')
parameter_df.to_csv(parameter_file, sep='\t', index=False)

print("✓ Parameter table created")
print("\n  Parameters to estimate:")
for _, row in parameter_df.iterrows():
    if row['estimate'] == 1:
        print(f"    • {row['parameterId']:15s} [{row['lowerBound']:.3f}, {row['upperBound']}]")
print("\n  🎯 Goal: Estimate δ (mRNA lifetime) from both datasets!")

## Step 2f: Create PEtab YAML Configuration

In [ ]:
yaml_content = f"""format_version: 1
parameter_file: parameters.tsv
problems:
  - sbml_files:
      - translation_model.xml
    measurement_files:
      - measurements.tsv
    condition_files:
      - conditions.tsv
    observable_files:
      - observables.tsv
"""

yaml_file = os.path.join(petab_dir, 'problem.yaml')
with open(yaml_file, 'w') as f:
    f.write(yaml_content)

print("✓ PEtab YAML configuration created")
print(f"\n✅ Complete PEtab problem defined in: {petab_dir}")
print("\n📦 PEtab files created:")
for f in ['problem.yaml', 'translation_model.xml', 'measurements.tsv', 
          'observables.tsv', 'conditions.tsv', 'parameters.tsv']:
    print(f"  • {f}")

---

# Part 3: AMICI Model Compilation

## Why AMICI?

**AMICI** = Advanced Multilanguage Interface to CVODES and IDAS
- Compiles SBML models to **efficient C++ code**
- Automatic **gradient computation** (adjoint sensitivity analysis)
- Orders of magnitude **faster** than pure Python ODE solvers
- Essential for large-scale parameter estimation

**Without AMICI**: Hours to days  
**With AMICI**: Minutes to hours

In [ ]:
import amici
import amici.petab
import amici.petab.conditions
import amici.petab.parameter_mapping
import amici.petab.simulations
from amici.petab.import_helpers import check_model

In [ ]:
print("Loading PEtab problem...")
petab_problem = petab.Problem.from_yaml(yaml_file)
petab.validate(yaml_file)

print("✓ PEtab problem loaded")
print(f"  • Estimated parameters: {petab_problem.n_estimated}")
print(f"  • Measurements: {len(petab_problem.measurement_df)}")
print(f"  • Conditions: {len(petab_problem.condition_df)}")

print("\nCompiling AMICI model (this may take 1-2 minutes)...")
print("  → Generating C++ code from SBML")
print("  → Computing symbolic derivatives")
print("  → Compiling shared library\n")

model_output_dir = os.path.join(petab_dir, 'amici_model')

hierarchical = True  # ← USER: Set to True for hierarchical noise model
problem = pypesto.petab.PetabImporter(
    petab_problem,
    hierarchical=hierarchical,
    model_name=f'translation_model',
).create_problem(force_compile=True)

---

# Part 4: pypesto Parameter Estimation

## Why pypesto?

**pypesto** = Parameter Estimation TOolbox
- **Multi-start optimization**: Avoids local minima
- **Profile likelihoods**: Assess parameter uncertainty
- **Multiple optimizers**: Scipy, Ipopt, Fides, etc.
- **Sampling methods**: MCMC, parallel tempering
- **Integrates** with AMICI for speed

### The Challenge:
High-dimensional, non-convex optimization landscape → Need robust multi-start approach

In [ ]:
print("Creating pypesto problem...")

problem.objective.amici_model.setAllStatesNonNegative()

print("✓ pypesto problem created")
print(f"  • Parameters to estimate: {problem.dim}")
print(f"  • Objective: Negative log-likelihood")
print(f"  • Using AMICI for fast simulation + gradients")

## Step 4: Run Multi-Start Optimization

**Strategy**: Start from multiple random initial guesses to find global optimum

**Note**: For teaching demo, we use 5 starts. In practice, use 50-100 starts.

In [ ]:
# Configure optimizer
maxiter = 5000  # ← USER: Increase for better convergence
n_starts = 1000  # ← USER: Increase for better statistics
n_procs = 4  # ← USER: Adjust based on your CPU cores

optimizer = pypesto.optimize.FidesOptimizer(verbose=False, options={'maxiter': maxiter})
engine = pypesto.engine.MultiProcessEngine(n_procs=n_procs)

result = pypesto.optimize.minimize(
    problem=problem,
    optimizer=optimizer,
    n_starts=n_starts,
    engine=engine,
    history_options = pypesto.HistoryOptions(trace_record=False, storage_file='optimization_history/n'+ str(n_starts) +'.hdf5'),
    filename='optimization_history/n'+ str(n_starts) +'.hdf5',
)

print(f"Running optimization with {n_starts} multi-starts...")
print("  (This may take a few minutes)\n")

In [ ]:

print("\n✓ Optimization complete!")
print(f"  Best objective value: {result.optimize_result.list[0]['fval']:.2f}")
print(f"  Successful starts: {len([r for r in result.optimize_result.list if r['exitflag'] > 0])} / {n_starts}")

---

# Part 5: Results Analysis

## Step 5a: Extract Parameter Estimates

In [ ]:
# Get best parameters
best_params_scaled = result.optimize_result.list[0]['x']
param_names = problem.x_names

# Unscale from log10
best_params = 10**best_params_scaled

# Create results dataframe
results_df = pd.DataFrame({
    'Parameter': param_names,
    'Estimated': best_params,
    'Unit': ['1/s', '1/s', '1/s', 's', 'a.u.', 'a.u.', 'a.u.'][:len(param_names)]
})

print("\n" + "="*70)
print(" "*20 + "PARAMETER ESTIMATION RESULTS")
print("="*70)
print(results_df.to_string(index=False))
print("="*70)

# Highlight key findings
delta_est = best_params[param_names.index('delta_param')]
mrna_halflife = np.log(2) / delta_est

print(f"\n🎯 KEY RESULT: mRNA half-life = {mrna_halflife:.2f} hours")
print(f"\n   This is BIOLOGICALLY MEANINGFUL:")
print(f"   • Determines therapeutic efficacy duration")
print(f"   • Informs dosing strategies for mRNA therapeutics")
print(f"   • ~7 hours typical for synthetic mRNA in mammalian cells")

## Step 5b: Visualize Optimization Convergence

In [ ]:
# Waterfall plot: shows convergence of multi-start optimization
fig = pypesto.visualize.waterfall(result, size=(10, 6))
plt.title('Multi-Start Optimization: Convergence to Global Optimum', fontsize=14, fontweight='bold')
plt.ylabel('Negative Log-Likelihood', fontsize=12)
plt.xlabel('Optimization Run (sorted)', fontsize=12)
plt.tight_layout()
plt.show()

print("\n📈 Waterfall Plot Interpretation:")
print("  • Each bar = one optimization run (starting point)")
print("  • Lower values = better fit to data")
print("  • Flat left side → converged to global optimum")
print("  • Rising right side → trapped in local minima")
print("  → Multi-start strategy successfully found global optimum!")

## Step 5c: Visualize Model Fits to Data

In [ ]:
# Simulate model with best parameters

return_dict = problem.objective(result.optimize_result.list[0].x, return_dict=True)
rdatas = return_dict['rdatas']
edatas = problem.objective.edatas
simulation = rdatas[0].y[:,0]
data = np.array(edatas[0].getObservedData())

In [ ]:
from pypesto.visualize.model_fit import _get_simulation_rdatas

amici_model = problem.objective.amici_model

species_to_plot = ['mRNA', 'Protein']

timepoints = np.linspace(start=0, stop=120000, num=1000)

simulation_rdatas = _get_simulation_rdatas(
    result=result,
    problem=problem,
    start_index = 0,
    simulation_timepoints=timepoints,
)

In [ ]:
# visualize model fit using simulation and data arrays
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(edatas[0].getTimepoints(), simulation, label=f'Simulation (Protein)')
plt.scatter(edatas[0].getTimepoints(), data, label=f'Data (Protein)', marker='o')

plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.title('Model Fit')
plt.show()

---

# Part 6: Conclusions from Fröhlich et al. (2018)

## What Did the Paper Discover?

### 1. **Problem**: Parameter Non-Identifiability
- From a *single* experiment (e.g., only eGFP), you **cannot uniquely estimate** both δ and γ
- Mathematical symmetry: swapping δ ↔ γ gives nearly identical fits
- **Solution**: Use *multiple* experiments with different protein degradation rates

### 2. **Method**: Nonlinear Mixed Effects (NLME) Modeling
- Standard two-stage (STS) approach: fit each cell independently → pool results
- **NLME approach** (what we implemented): 
  - Fit population + individual cells *simultaneously*
  - Share parameters across experiments (δ, k) while allowing differences (γ)
  - Breaks parameter symmetry → unique estimates!

### 3. **Biological Insight**: Ribosome Rate-Limitation
- Model (i) (what we used): First-order translation kinetics
- **Model (ii) performed better**: Explicit ribosome binding to mRNA
- Finding: **Ribosomes are rate-limiting** in translation
- Mechanistic understanding > phenomenological fit

### 4. **Robustness**: Handling Batch Effects
- Even with standardized assays, batch-to-batch variability exists
- NLME can **identify sources** of batch effects (e.g., transfection efficiency m₀)
- Critical for multi-batch integration

---

# Part 7: Extensions - Modern AI/ML Approaches

## Beyond Mechanistic Models: When Do We Need More?

### Limitations of Pure Mechanistic Models:
1. **Incomplete knowledge**: Don't know all reactions/interactions
2. **Missing components**: Unmeasured species affect dynamics
3. **Complex kinetics**: True mechanisms may be more complex than mass-action
4. **Spatial effects**: Well-mixed assumption breaks down

---

## Extension 1: **Universal Differential Equations (UDEs)**

### Concept: Hybrid Mechanistic + Neural Network

**Keep what you know, learn what you don't:**

$$\frac{d[\text{mRNA}]}{dt} = -\delta [\text{mRNA}] + \underbrace{NN_1(\text{mRNA, GFP, t})}_{\text{unknown regulation}}$$

$$\frac{d[\text{GFP}]}{dt} = \underbrace{k[\text{mRNA}]}_{\text{known: translation}} - \gamma [\text{GFP}]$$

### When to Use UDEs:
- ✅ Partial mechanistic knowledge exists
- ✅ Suspect additional unmeasured factors
- ✅ Want interpretable parameters (δ, k, γ) + flexible corrections
- ✅ Have enough data to constrain neural network

### Tools:
- **Julia**: `DifferentialEquations.jl` + `DiffEqFlux.jl`
- **Python**: `torchdiffeq`, `diffrax` (JAX)

### Example Use Case:
"We know mRNA degrades, but there might be feedback regulation we don't understand"

---

## Extension 2: **Neural ODEs**

### Concept: Fully Data-Driven Dynamics

**Learn the entire dynamic law:**

$$\frac{d\mathbf{x}}{dt} = NN(\mathbf{x}, t; \theta)$$

where $\mathbf{x} = [\text{mRNA, GFP, ...}]$ and $NN$ is a neural network.

### When to Use Neural ODEs:
- ✅ Little/no mechanistic knowledge
- ✅ Complex systems (many interacting species)
- ✅ Phenomenological prediction is sufficient
- ❌ Want interpretable parameters → Not ideal
- ❌ Limited data → Risk of overfitting

### Tools:
- **Python**: `torchdiffeq` (PyTorch), `diffrax` (JAX)
- **Julia**: `DiffEqFlux.jl`

### Example Use Case:
"We want to predict cell fate from time-lapse data, but don't know the underlying mechanisms"

---

## Extension 3: **Mixed Effects + Neural ODEs**

### Combining Population Variability + Data-Driven Dynamics

- **NLME** (what we used): Models population distribution of *mechanistic* parameters
- **Neural ODE + NLME**: Model population distribution of *neural network weights*

### When to Use:
- Large single-cell datasets with heterogeneity
- Want to capture both population structure AND complex dynamics
- Example: Cancer cell response to therapy (heterogeneous + complex)

---

## Comparison Summary:

| Approach | Mechanistic Knowledge | Interpretability | Data Requirements | Best For |
|----------|----------------------|------------------|-------------------|----------|
| **Mechanistic ODE** (today) | Full | ⭐⭐⭐⭐⭐ | Moderate | Well-understood systems |
| **Universal ODE (UDE)** | Partial | ⭐⭐⭐⭐ | Moderate-High | Hybrid: known + unknown |
| **Neural ODE** | None | ⭐⭐ | High | Black-box prediction |
| **NLME + Mechanistic** (Fröhlich) | Full | ⭐⭐⭐⭐⭐ | Moderate | Single-cell heterogeneity |

---

## For mRNA Translation: Which Approach?

1. **Start with mechanistic** (what we did) ✅
   - Strong prior knowledge of translation process
   - Interpretable parameters (δ, k, γ)
   - Sufficient for most questions

2. **Add UDE if needed**:
   - Residual analysis shows systematic deviations
   - Suspect unmeasured regulatory factors
   - Example: stress response affecting translation

3. **Neural ODE if...**:
   - Predicting expression in new cell types
   - Don't care about mechanism, just prediction
   - Have massive datasets (>10,000 cells)

---

## PEtab for Hybrid Models?

**Current status**: PEtab focuses on mechanistic models  
**Future**: Community discussing standards for hybrid/UDE models

---

---

# Summary: The pypesto → AMICI → PEtab Pipeline

## What We Demonstrated:

1. **PEtab**: Standardized, reproducible problem definition
   - Model (SBML) + Data (tables) + Parameters (bounds, priors)
   - Shareable, version-controlled, tool-agnostic

2. **AMICI**: Fast ODE simulation
   - Compiled C++ code → 10-100× speedup
   - Automatic gradients → enables efficient optimization

3. **pypesto**: Robust parameter estimation
   - Multi-start optimization → avoids local minima
   - Profile likelihoods → quantify uncertainty
   - Integrates seamlessly with AMICI + PEtab

---

## When to Use This Approach:

✅ **You have**:
- Mechanistic ODE model
- Time-series measurements (bulk or single-cell)
- Unknown parameters to estimate

✅ **You want**:
- Quantitative parameter estimates
- Uncertainty quantification
- Model selection / hypothesis testing
- Reproducible, shareable analysis

---

## Key Advantages:

- **Speed**: AMICI enables gradient-based optimization (essential for >10 parameters)
- **Standardization**: PEtab connects to BioModels, benchmark collections
- **Scalability**: Works for 10-100+ parameter models
- **Community**: Active development, good documentation, growing user base

---

## Next Steps for Your Research:

1. **Try it on your own data!**
   - Replace our data files with yours
   - Modify SBML model for your system

2. **Explore extensions**:
   - Profile likelihoods (parameter uncertainty)
   - MCMC sampling (Bayesian inference)
   - Model selection (compare hypotheses)

3. **Join the community**:
   - GitHub: github.com/ICB-DCM/pyPESTO, github.com/AMICI-dev/AMICI
   - Documentation: pypesto.readthedocs.io
   - PEtab: petab.readthedocs.io

---

## Resources:

- **Original Paper**: Fröhlich et al. (2018) npj Sys Bio & Apps https://doi.org/10.1038/s41540-018-0079-7
- **Data**: https://doi.org/10.5281/zenodo.1228898
- **pypesto**: https://pypesto.readthedocs.io
- **AMICI**: https://amici.readthedocs.io
- **PEtab**: https://petab.readthedocs.io
- **Benchmark Models**: https://github.com/Benchmarking-Initiative/Benchmark-Models-PEtab

---

## Questions?

**Thank you!** 🎉

---